In [1]:
##import all the required libraries
import pandas as pd
import numpy as np
import os
from collections import Counter
from gensim.models import word2vec
import sklearn
from sklearn import manifold
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora
from gensim import models
from gensim import similarities
import heapq

In [3]:
df_ingre = pd.read_csv('recipe_dataset/db/recipe_ingre.csv')
df_basic = pd.read_csv('recipe_dataset/db/recipe_basic.csv', encoding="cp949")

df_tmp=df_ingre[['RECIPE_ID','IRDNT_NM']]
df_tmp_arr=[[]for i in range(538)]
num=0
for i in df_tmp['IRDNT_NM'] :
    df_tmp_arr[df_tmp['RECIPE_ID'][num]].append(i)
    num+=1
    
df_tmp_arr.pop(0)
num=0
for i in df_tmp_arr :
    df_tmp_arr[num] = list(set(df_tmp_arr[num]))
    num+=1
    
ingre_dict = {}

for i in range(1, 538, 1):
    ingre_dict[str(i)] = df_tmp_arr[i-1]

In [4]:
bags_of_words = [ Counter(IRDNT_NM) for IRDNT_NM in df_tmp_arr ]

In [6]:
#Find sum of every ingredient using Counter()
sumbags = sum(bags_of_words, Counter())


# Finally, plot the 10 most used ingredients
clean_df = pd.DataFrame.from_dict(sumbags, orient='index').reset_index()
clean_df = clean_df.rename(columns={'index':'IRDNT_NM', 0:'count'})

top_ing = clean_df.sort_values('count', ascending=False)

In [7]:
ingr_only_dict = clean_df['IRDNT_NM'].to_dict()

In [8]:
#Get all clean ingredients in list format per recipe
ingr_list = []
for IRDNT_NM in df_tmp_arr:
    ingr_list.append(IRDNT_NM)

In [10]:
##Get clean ingredients to be used as an input for word2vec model to identify ingredient similarity.

ingr_clean_df = pd.DataFrame({'IRDNT_NM':ingr_list})

In [12]:
#Create a dictionary for all the ingredients in the recipe list

dictionary = corpora.Dictionary(ingr_list)

In [13]:
#Applying doc2bow on the dictionary of ingredients, which converts the ingredient to a number in every recipe
#This input format is needed for TfIdfmodel
bow_corpus = [dictionary.doc2bow(text) for text in ingr_list]

In [14]:
# train the model
tfidf = models.TfidfModel(bow_corpus)

In [15]:
corpus_tfidf = tfidf[bow_corpus]

In [16]:
#Use similarities library from gensim to get the cosine similarity of the tfidf results

index = similarities.MatrixSimilarity(tfidf[bow_corpus])
index.save('ingr.index')
index = similarities.MatrixSimilarity.load('ingr.index')

sims = index[corpus_tfidf]
sims_list = [(i,j) for i,j in enumerate(sims)]

#Creating a list to hold the cosine similarity results for tfidf
tf_idf_list = []

for i,j in enumerate(sims_list):
    tf_idf_list.append(sims_list[i][1])

In [17]:
#Create recipe dict- to be used in creating dataframe in next step - used to decode recipe id
recipe_dict = {k: v for k, v in enumerate(df_basic.RECIPE_NM_KO)}

In [18]:
#Use cosine similarity results to get the top 10 similar recipes for every recipe.
tf_idf_top  = []
similar_recipes_df = pd.DataFrame([])
same_item = []

#Get only top 11 largest values from the tf_idf_list - 1 recipe will be the same as itself (hence 12)
for i,item in enumerate(tf_idf_list):
    tf_idf_top.append(heapq.nlargest(11,enumerate(item), key=lambda x: x[1]))

#Remove the recipe value with 1.0 similarity - since it is the same recipe
for i,list_item in enumerate(tf_idf_top):
    for j,k in enumerate(list_item):
        if tf_idf_top[i][j][1] != 1.0:
            similar_recipes_df = similar_recipes_df.append(pd.DataFrame({'Similar_Recipe_ID': recipe_dict.get(tf_idf_top[i][j][0]),'TF-IDF Value': tf_idf_top[i][j][1],'Recipe_ID': recipe_dict.get(i)}, index=[0]), ignore_index=True)

In [19]:
similar_recipes_df.to_csv('similar_recipes_top_10_tf_idf.csv',encoding='utf-8')

In [20]:
similar_recipes_df = pd.read_csv('similar_recipes_top_10_tf_idf.csv')
similar_recipes_df = similar_recipes_df[similar_recipes_df['Recipe_ID'] != similar_recipes_df['Similar_Recipe_ID']]

In [21]:
similar_recipes_df

,Unnamed: 0,Similar_Recipe_ID,TF-IDF Value,Recipe_ID
1,1,상추겉절이비빔밥,0.459678,나물비빔밥
2,2,사색나물,0.451160,나물비빔밥
3,3,돌솥비빔밥,0.417394,나물비빔밥
4,4,콩나물잡채,0.390625,나물비빔밥
5,5,묵과양념장,0.387687,나물비빔밥
...,...,...,...,...
5486,5486,잡채밥,0.212544,콩비지동그랑땡
5487,5487,배추속댓국,0.207736,콩비지동그랑땡
5488,5488,중국식볶음밥,0.195908,콩비지동그랑땡
5489,5489,제육배추찜,0.189969,콩비지동그랑땡


In [22]:
similar_recipes_df = similar_recipes_df[similar_recipes_df['Recipe_ID'] != similar_recipes_df['Similar_Recipe_ID']]

In [23]:
#Create cosine similarity matrix for all recipes 27637*27637
#Since this is a huge matrix, the top 10 similar recipe logic is a better option.

names = [i for i in range(1,len(tf_idf_list))]
final_df = pd.DataFrame.from_dict(zip(names,tf_idf_list))

In [24]:
df_basic['RECIPE_ID'] = df_basic.index+1
recipe_name_df = df_basic[['RECIPE_ID','RECIPE_NM_KO']]
final_df['RECIPE_ID'] = final_df.index+1

recipe_tf_idf_df = final_df.merge(recipe_name_df,how='left', left_on='RECIPE_ID', right_on='RECIPE_ID')

In [25]:
#Create a list from tfidf results
#This will be used to identify ingredient importance within every recipe

corpus_list = []
for doc in corpus_tfidf:
    corpus_list.append(doc)

In [26]:
#Create a flat list to eliminate repetition of ingredients and create a dict to hold the results

flat_list = []
for sublist in ingr_list:
    for item in sublist:
        if item not in flat_list:
            flat_list.append(item)
ing_dict =  {k: v for k, v in enumerate(flat_list)}

In [27]:
#Create a dataframe with tf-idf values per ingredient for every recipe.
corpus_df = pd.DataFrame([])

for i,list_item in enumerate(corpus_list):
    for j,k in enumerate(list_item):
        corpus_df = corpus_df.append(pd.DataFrame({'IRDNT_NM': dictionary.get(corpus_list[i][j][0]),'TF-IDF Value': corpus_list[i][j][1],'Recipe_ID': i}, index=[0]), ignore_index=True)

In [28]:
df_basic['RECIPE_ID'] = df_basic.index
recipe_tf_idf_df = corpus_df.merge(df_basic,how='left', left_on='Recipe_ID', right_on='RECIPE_ID')
recipe_tf_idf_df = recipe_tf_idf_df[['Recipe_ID','RECIPE_NM_KO','IRDNT_NM','TF-IDF Value']]
tf_idf_sorting = recipe_tf_idf_df.sort_values(by=['Recipe_ID', 'IRDNT_NM'], axis=0)
tf_idf_sorting.set_index('Recipe_ID', inplace=True)
tf_idf_sorting.index = tf_idf_sorting.index+1
tf_idf_sorting.reset_index(inplace=True)
# tf_idf_sorting.to_csv('recipe_dataset/tf_idf_sorting.csv', encoding="utf-8-sig")

KeyError: "['IRDNT_NM'] not in index"

In [ ]:
tf_idf_df = pd.read_csv('recipe_dataset/tf_idf_sorting.csv')
tf_idf_df = tf_idf_df.drop(['Unnamed: 0'], axis = 1)
tf_idf_df = pd.concat([tf_idf_df, df_ingre["SCORE"]], axis = 1)
tfidf_weight = pd.DataFrame(tf_idf_df['TF-IDF Value'].mul(tf_idf_df['SCORE'], axis=0, level=None, fill_value=None))
tf_idf_df.insert(3, 'TF-IDF_WEIGHT',tfidf_weight)
tf_idf_df

In [ ]:
# tf_idf_df.to_csv('tf_idf_df.csv', encoding="utf-8-sig")

In [ ]:
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=1234)